In [16]:
%pwd

'x:\\CampusX\\projects\\end-to-end_laptop_project\\research'

In [17]:
import os
os.chdir("../")

In [18]:
%pwd

'x:\\CampusX\\projects\\end-to-end_laptop_project'

In [19]:
from dataclasses import dataclass
from pathlib import Path

In [20]:
@dataclass(frozen=True)
class MissingValueImputationConfig:
    root_dir: Path
    after_cleaning: Path
    filled_data_path: Path

In [21]:
from laptop_ml.constants import *
from laptop_ml.utils.common import read_yaml,create_directories

In [22]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)
        create_directories([self.config.artifacts])

    def get_missing_value_imputation_config(self)->MissingValueImputationConfig:
        config=self.config.missing_value_imputation
        create_directories([config.root_dir])

        missing_value_imputation_config=MissingValueImputationConfig(
            root_dir=config.root_dir,
            after_cleaning=config.after_cleaning,
            filled_data_path=config.filled_data_path
        )
        return missing_value_imputation_config
        

In [23]:
import pandas as pd
import numpy as np

In [24]:
class MissingValueImputation:
    def __init__(self, config: MissingValueImputationConfig):
        self.config=config
        self.df=pd.read_csv(self.config.after_cleaning)

    def convert_display_size(self,x):
        if x>11 and x<=14:
            return "small"
        elif x>14 and x<16:
            return "medium"
        else:
            return "large"
        
    def convert_weight(self,x):
        if x<=1.5:
            return "lite"
        elif x>1.5 and x<2.0:
            return "medium"
        else:
            return "heavy"
        
    def thickness_convert(self,x):
        if x<=16:
            return "slim"
        elif x>16 and x<23:
            return "medium"
        else:
            return "thick"
        
    def export_csv(self,df):
        df.to_csv(self.config.filled_data_path)

        
    def missing_value_imputation(self):
        self.df["brand"]=self.df["brand"].str.lower()

        self.df["display_size"]=self.df["display_size"].apply(self.convert_display_size)

        mean_weight=self.df.groupby("display_size")["weight"].mean()

        for i in self.df[self.df["weight"].isnull()].index:
            if self.df.loc[i,"display_size"]=="large":
                self.df.loc[i,"weight"]=mean_weight[0]
            elif self.df.loc[i,"display_size"]=="medium":
                self.df.loc[i,"weight"]=mean_weight[1]
            else:
                self.df.loc[i,"weight"]=mean_weight[2]

        
        self.df["weight"]=self.df["weight"].apply(self.convert_weight)

        mean_thickness=self.df.groupby("weight")["thickness"].mean()

        for i in self.df[self.df["thickness"].isnull()].index:
            if self.df.loc[i,"weight"]=="heavy":
                self.df.loc[i,"thickness"]=mean_thickness[0]
            elif self.df.loc[i,"weight"]=="lite":
                self.df.loc[i,"thickness"]=mean_thickness[1]
            else:
                self.df.loc[i,"thickness"]=mean_thickness[2]

        self.df["thickness"]=self.df["thickness"].apply(self.thickness_convert)

        for i in self.df[self.df["utility"].isnull()].index:
            if self.df.loc[i,"weight"]=="heavy":
                self.df.loc[i,"utility"]="Gaming"
            else:
                self.df.loc[i,"utility"]="Performance"

        for i in self.df[self.df["warranty"].isnull()].index:
            if self.df.loc[i,"price"]<=90000:
                self.df.loc[i,"warranty"]=1
            elif self.df.loc[i,"price"]<=135000:
                self.df.loc[i,"warranty"]=2
            else:
                self.df.loc[i,"warranty"]=3

        for i in self.df[self.df["num_of_cell"].isnull()].index:
            if self.df.loc[i,"price"]<=50000:
                self.df.loc[i,"num_of_cell"]=2
            elif self.df.loc[i,"price"]<=80000:
                self.df.loc[i,"num_of_cell"]=3
            elif self.df.loc[i,"price"]<=150000:
                self.df.loc[i,"num_of_cell"]=4
            else:
                self.df.loc[i,"num_of_cell"]=6

        mean_battery=self.df.groupby("num_of_cell")["battery_capacity"].mean()

        for i in self.df[self.df["battery_capacity"].isnull()].index:
            if self.df.loc[i, "num_of_cell"] == 2.0:
                self.df.loc[i, "battery_capacity"] = mean_battery[2.0]
            elif self.df.loc[i, "num_of_cell"] == 3.0:
                self.df.loc[i, "battery_capacity"] = mean_battery[3.0]
            elif self.df.loc[i, "num_of_cell"] == 4.0:
                self.df.loc[i, "battery_capacity"] = mean_battery[4.0]
            else:
                self.df.loc[i, "battery_capacity"] = mean_battery[6.0]

        self.df["core"].fillna(self.df["core"].mode()[0],inplace=True)

        for i in self.df[self.df["thread"].isnull()].index:
            if self.df.loc[i,"core"]==8:
                self.df.loc[i,"thread"]=16
            elif self.df.loc[i,"core"]==14:
                self.df.loc[i,"thread"]=20
            elif self.df.loc[i,"core"]==6:
                self.df.loc[i,"thread"]=12
            elif self.df.loc[i,"core"]==24:
                self.df.loc[i,"thread"]=32
            elif self.df.loc[i,"core"]==12:
                self.df.loc[i,"thread"]=16
            elif self.df.loc[i,"core"]==10:
                self.df.loc[i,"thread"]=12
            else:
                self.df.loc[i,"thread"]=self.df.loc[i,"core"]

        value=self.df[self.df["cache"]=="Smart Cache Cache"]
        self.df.loc[value.index,"cache"]=np.nan
        self.df["cache"].fillna(self.df["cache"].mode()[0],inplace=True)
        self.df["cache"]=self.df["cache"].astype("float")

        self.df["pixel_height"].fillna(self.df["pixel_height"].mode()[0],inplace=True)
        self.df["pixel_width"].fillna(self.df["pixel_width"].mode()[0],inplace=True)

        val_list=["intel","mediatek","amd","apple"]
        for i in self.df[self.df["processor_gen"].str.lower().isin(val_list)].index:
            if self.df.loc[i,"processor_gen"]=="Apple":
                self.df.loc[i,"processor_gen"]="2nd"
            elif self.df.loc[i,"processor_gen"]=="Intel":
                self.df.loc[i,"processor_gen"]="3rd"
            elif self.df.loc[i,"processor_gen"]=="intel":
                self.df.loc[i,"processor_gen"]="3rd"
            elif self.df.loc[i,"processor_gen"]=="AMD":
                self.df.loc[i,"processor_gen"]="7th"
            elif self.df.loc[i,"processor_gen"]=="MediaTek":
                self.df.loc[i,"processor_gen"]="1st"

        self.df["processor_gen"]=self.df["processor_gen"].str.slice(0,-2)
        self.df["processor_gen"]=self.df["processor_gen"].astype("float")

        for i in self.df["ppi"]:
            if i<=141:
                self.df["ppi"].replace(i,"low",inplace=True)
            elif i<=157:
                self.df["ppi"].replace(i,"medium",inplace=True)
            elif i<=250:
                self.df["ppi"].replace(i,"high",inplace=True)
            else:
                self.df["ppi"].replace(i,"very_high",inplace=True)

        self.df["processor"]=self.df["processor_brand"]+" "+self.df["processor_model"]+" "+self.df["processor_gen"].astype(str)
        self.df["brand"]=self.df["brand"].apply(lambda x:x if self.df["brand"].value_counts()[x]>4 else "others")
        self.df["processor"]=self.df["processor"].apply(lambda x:x if self.df["processor"].value_counts()[x]>6 else "others")
        
        self.export_csv(self.df)
    

In [25]:
try:
    config=ConfigurationManager()
    missing_value_config=config.get_missing_value_imputation_config()
    missing_value=MissingValueImputation(config=missing_value_config)
    missing_value.missing_value_imputation()
except Exception as e:
    raise e

[2024-08-22 23:06:16,210: INFO :common :yaml file: config\config.yaml loaded successfully]
[2024-08-22 23:06:16,233: INFO :common :yaml file: params.yaml loaded successfully]
[2024-08-22 23:06:16,236: INFO :common :created directory at: root]
[2024-08-22 23:06:16,238: INFO :common :created directory at: artifacts/missing_value_imputation]


C:\Users\harsh\AppData\Local\Temp\ipykernel_13024\4069275552.py:43: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.df.loc[i,"weight"]=mean_weight[0]
C:\Users\harsh\AppData\Local\Temp\ipykernel_13024\4069275552.py:45: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.df.loc[i,"weight"]=mean_weight[1]
C:\Users\harsh\AppData\Local\Temp\ipykernel_13024\4069275552.py:45: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.df.loc[i,"w